# Homework 2

## Problem 1

In [88]:
from random import *
from math import *
from functools import reduce
import operator

spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

class SpamFilter():
    
    def __init__(self, good_corpus, bad_corpus, email=[]):
        self.email = email
        self.good_corpus = good_corpus
        self.bad_corpus = bad_corpus
        self.good_table = token_count(good_corpus)
        self.bad_table = token_count(bad_corpus)

    def token_count(self):
        '''takes corpus and returns dictionary with each token of the corpus
        as a key and the number of times the token has appeared as a value'''
        words = {}
        for mail in corpus:
            for token in mail:
                if token not in words:
                    words[token] = 1
                else:
                    words[token] += 1
        return words  
        
    def gen_mail(self, isGood=bool(getrandbits(1)), length=20):
        '''returns email randomly generated'''
        if isGood:
            email = []
            for i in range(length):
                email.append(choice(choice(self.good_corpus)))
            self.email = email
            return email
        else:
            email = []
            for i in range(length):
                email.append(choice(choice(self.bad_corpus)))
            self.email = email
            return email
        
    def pr_token(self, word):
        '''calculates probability token is a spam token'''
        ngood = len(self.good_corpus)
        nbad = len(self.bad_corpus)
        try:
            g = 2 * self.good_table[word]
        except KeyError:
            g = 0
        try:
            b = self.bad_table[word]
        except KeyError:
            b = 0
        if (g + b) >= 1:
            return max( 0.01, min( 0.99, float((min(1, b / nbad))/(min(1, g / ngood) + min(1, b / nbad)))))
        else:
            return None
        
        
    def pr_mail(self, ntokens=5):
        '''takes email; calculate probability an email is spam; return confidence an email is spam'''
        interest_table = {}
        mail_table = {}
        for token in self.email:
            mail_table[token] = self.pr_token(token)
            token_interest = fabs(0.50 - self.pr_token(token))
            interest_table[token] = token_interest
        interesting_tokens = sorted(interest_table, key=interest_table.get, reverse=True)[:ntokens]
        
        interesting_token_values = []
        for token in interesting_tokens:
            if token in interesting_tokens:
                interesting_token_values.append(mail_table[token])
        
        polynom = []
        for v in interesting_token_values:
            polynom.append(1 - v)
        
        confidence = prod(interesting_token_values) / (prod(interesting_token_values) + prod(polynom))
        return confidence
            
        
filter0 = SpamFilter(good_corpus=ham_corpus, bad_corpus=spam_corpus)
email = filter0.gen_mail(isGood=False, length=10)
print(email)
print(filter0.pr_mail(3))
sample = ['do', 'do', 'do', 'do', 'do', 'do', 'i', 'spam', 'eggs']
filter1 = SpamFilter(email=sample, good_corpus=ham_corpus, bad_corpus=spam_corpus)
print(filter1.pr_mail(3))


['that', 'like', 'I', 'I', 'am', 'I', 'not', 'spam', 'I', 'not']
0.99999896939091
0.009999999999999992


This approach is Baysian because of the 
`prod(interesting_token_values) / (prod(interesting_token_values) + prod(polynom))` 
expression. The confidence that an email is spam is calculated by using Bayes rule by multiplying the probabilities that a 